In [1]:
using Random, Distributions, LinearAlgebra, FunctionOperators, ToeplitzMatrices, Formatting

In [2]:
import Base.size
function Base.size(FO::FunctionOperator, d::Int)
    @assert d in [1, 2]
    prod(d == 1 ? FO.outDims : FO.inDims)
end

# Robust Principal Component Analysis?
*by Emmanuel J. Candès, Xiaodong Li, Yi Ma, and John Wright*  
https://arxiv.org/pdf/0912.3599.pdf

## 4.1 Exact recovery from varying fractions of error

[...] We consider square matrices of varying dimension $n = 500, \ldots , 3000$. We generate a rank-$r$ matrix $L_0$ as a product $L_0 = XY^∗$ where $X$ and $Y$ are $n \times r$ matrices with entries independently sampled from a $\mathcal{N}(0,1/n)$ distribution. $S_0$ is generated by choosing a support set $\Omega$ of size $k$ uniformly at random, and setting $S_0 = \mathcal{P}_\Omega E$, where $E$ is a matrix with independent Bernoulli $\pm 1$ entries. [...]

In [3]:
function generateLowRankComponent_Candes(n, r, dType)
    𝓝 = Normal(0, 1/n)
    X = rand(𝓝, dType, n, r)
    Y = rand(𝓝, dType, n, r)
    L₀ = X * Y'
end

generateLowRankComponent_Candes (generic function with 1 method)

In [4]:
function generateSparseComponent_Candes(n, k, dType)
    Ω = randperm(n*n)[1:k] # indices of non-zero elements
    𝓑 = Binomial()
    E = rand(𝓑, n, n)
    E[E.==0] .= -1
    S₀ = zeros(n, n)
    S₀[Ω] = E[Ω]
    S₀
end

generateSparseComponent_Candes (generic function with 1 method)

# Christian's Code

This function randomly samples a $(d_1 \times d_2)$ sparse matrix with ones at $m$ randomly chosen
coordinates (uniform without replacement). The output matrix has at least $r$ non-zero entries
in each row and each column, where $r$ is a specified positive integer. The number of ones in the
output matrix is exactly $m$.

In [5]:
function generateΦ(d₁, d₂, r, m)
    @assert max(d₁, d₂) * r ≤ m
    @assert m ≤ d₁ * d₂
    @assert r ≤ d₁
    @assert r ≤ d₂
    
    # generate a square matrix where each row and each column has exactly r ones
    initial = Circulant([fill(1, r)..., fill(0, min(d₁, d₂) - r)...])
    
    # Extend that matrix to a d₁×d₂ matrix where each row and each column has at least r ones
    # That is accomplished by repeating the "initial" matrix and then cropping
    if d₁ < d₂
        M = repeat(initial, outer = (1, ceil(Int, d₂ / d₁)))
    elseif d₁ > d₂
        M = repeat(initial, outer = (ceil(Int, d₁ / d₂), 1))
    else
        M = initial
    end
    M = M[1:d₁, 1:d₂]
    
    # Randomly switch zeros to ones until exactly m number of ones are in the matrix
    zero_places = findall(M .== 0)
    number_of_missing_ones = m - (d₁*d₂ - length(zero_places))
    number_of_missing_ones > 0 && (M[shuffle(zero_places)[1:number_of_missing_ones]] .= 1)
    
    # Then randomize matrix by permutating rows and columns a couple times
    for i in 1:10
        M .= M[shuffle(1:end), :] # shuffle rows
        M .= M[:, shuffle(1:end)] # shuffle columns
    end
    
    M
end

generateΦ (generic function with 1 method)

In [6]:
function maskToMatrix(Φᴹ)
    m = convert(Int, sum(Φᴹ))
    d₁, d₂ = size(Φᴹ)

    Φ = zeros(m, length(Φᴹ))
    non_zero_places = findall(vec(Φᴹ) .== 1)
    for i in 1:m
        Φ[i, non_zero_places[i]] = 1
    end
    return Φ
end

maskToMatrix (generic function with 1 method)

In [7]:
function generateLowRankComponent_Christian(d₁, d₂, r, dType)randn()
    𝓝 = Normal(0, 1)
    U, S, V = rand(𝓝, dType, d₁, r), Diagonal(rand(𝓝, real(dType), r)), rand(𝓝, dType, d₂, r)
    L₀ = U * S * V'
end

generateLowRankComponent_Christian (generic function with 1 method)

In [8]:
import Random.rand
rand(N::Normal, T::Type{X}, dims...) where {X<:AbstractFloat} = T.(rand(N,  dims...))
rand(N::Normal, T::Type{Complex{X}}, dims...) where {X} =
    Complex{X}.(rand(N, dims...)) + Complex{X}.(rand(N, dims...))im

rand (generic function with 167 methods)

In [52]:
module DebugTableModule

import Formatting
export DebugTable, printRow

mutable struct DebugTable
    columns::Vector{Union{Tuple{String,Function}, Tuple{String,Function,Int},
                    Tuple{String,Nothing}, Tuple{String,Nothing,Int}}}
    defaultWidth::Int64
    defaultPrecision::Int64
    headAlreadyPrinted::Bool
end

DebugTable(tuples...; defaultWidth = 8, defaultPrecision = 3, headAlreadyPrinted = false) =
    DebugTable([e for e in tuples], defaultWidth, defaultPrecision, headAlreadyPrinted)

processColumnWidth(columns, defaultWidth) = begin
    for (i,columnTuple) in enumerate(columns)
        if length(columnTuple) < 3
            columns[i] = columnTuple..., max(length(columnTuple[1]), defaultWidth)
        elseif length(columnTuple[1]) > columnTuple[3]
            columns[i] = columnTuple[1], columnTuple[2], length(columnTuple[1])
        end
    end
    columns
end

printLine(columns, first, middle, last) = begin
    print(first)
    for (i,(_,_,cnw)) in enumerate(columns)
        print(repeat("─", cnw + 2))
        i < length(columns) && print(middle)
    end
    println(last)
end

center(str, width) = repeat(" ", ceil(Int, (width - length(str))/2)) * str *
    repeat(" ", floor(Int, (width - length(str))/2))

printColumnNames(columns) = begin
    for columnTuple in columns
        print("│ ", center(columnTuple[1], columnTuple[3]), " ")
    end
    println("│")
end

printHeader(DT::DebugTable) = begin
    processColumnWidth(DT.columns, DT.defaultWidth)
    printLine(DT.columns, "┌", "┬", "┐")
    printColumnNames(DT.columns)
    printLine(DT.columns, "├", "┼", "┤")
    DT.headAlreadyPrinted = true
end

printValue(value::Int, width, precision) =
    abs(value > 10^width) ?
        printValue(float(value), width, precision) : 
        Formatting.sprintf1("%$(width)d", value)
printValue(value::Real, width, precision) =
    Formatting.sprintf1(abs(value) < 10. ^ -(precision-1) || abs(value) > 10^(width-precision-1) ?
            "%$width.$(precision-1)e" : "%$width.$(precision)f", value)

function printRow(DT::DebugTable, args...; last::Bool = false)
    DT.headAlreadyPrinted || printHeader(DT)
    for (name, expr, width) in DT.columns
        positionInArgs = findfirst(x -> x[1] == name, args)
        value = positionInArgs !== nothing ? args[positionInArgs][2] : expr()
        print("│ ", printValue(value, width, DT.defaultPrecision), " ")
    end
    println("│")
    last && printLine(DT.columns, "└", "┴", "┘")
end

end

Main.DebugTableModule

In [53]:
using Main.DebugTableModule